In [1]:
using Laplacians


In [37]:
include("../devel/kmpTest.jl")
include("../src/KMPSolver.jl")

stretchSample (generic function with 1 method)

In [40]:
n = 100000
a = chimera(n)
la = lap(a)
b = randn(n);
b = b - mean(b);

#t = akpw(a)
#heavy = makeHeavy(a,t)
#lheavy = lap(heavy);

A test of the solver code, with some param settings thrown in

In [41]:
@time f = KMPLapSolver(a,params=KMPparams(1/10,6,1.3,200),maxits=10)
@time x = f(b)
@show norm(la*x-b)/norm(b);

level : 0
50132, 100000
level : 1
5290, 19351
level : 2
549, 2165
level : 3
40, 157
  1.726749 seconds (2.40 M allocations: 306.944 MB, 7.89% gc time)
  1.847639 seconds (264.05 k allocations: 1.156 GB, 15.34% gc time)
norm(la * x - b) / norm(b) = 0.8111931026764336


A comparison with our old solver

In [42]:
@time fa = lapWrapSolver(augTreeSolver,la,tol=1e-2,maxits=1000)
@time xa = fa(b)
@show norm(la*xa-b)/norm(b);


  0.756275 seconds (301.65 k allocations: 188.274 MB, 12.91% gc time)
  1.148191 seconds (3.53 k allocations: 853.815 MB, 7.71% gc time)
norm(la * xa - b) / norm(b) = 0.009760812394754752


To understand what's going on, let's look at how quickly it solves the precon system.


In [43]:
params=KMPparams(1/10,6,1.3,200)
heavy = makeHeavy(a, params)
lh = lap(heavy)
@time fsub =  KMPLapPrecon(a, akpw(a), params)
@time xh = fsub(b)
@show norm(lh*xh-b)/norm(b);

level : 0
50132, 100000
level : 1
5482, 19940
level : 2
550, 2160
level : 3
42, 164
  1.100389 seconds (1.31 M allocations: 231.301 MB, 10.46% gc time)
  0.236276 seconds (23.99 k allocations: 107.939 MB, 36.81% gc time)
norm(lh * xh - b) / norm(b) = 0.5094250259138964


In [46]:
#@time f = KMPLapSolver(a,params=KMPparams(1/10,6,1.3,200),maxits=30)
@time x = f(b)
@show norm(la*x-b)/norm(b);

  5.116178 seconds (743.95 k allocations: 3.327 GB, 11.78% gc time)
norm(la * x - b) / norm(b) = 0.21119848240431538


In [45]:
marked = 0 + (rand(n) .< 0.6)
b = randn(n)
b = b - mean(b)
lt = lap(t)
lt[1,1] = lt[1,1] + 1
x = recTest12x(t, b, marked)
norm(lt*x-b);


LoadError: LoadError: went of the end of the kid list without finding node to elim from
while loading In[45], in expression starting on line 6

5.515545 seconds (744.05 k allocations: 3.327 GB, 11.30% gc time)
norm(la * x - b) / norm(b) = 0.21378296018567286


In [ ]:
@time F = factorize(lt)

@time y = F \ b

norm(y - x)

In [ ]:
elims1, elims2 = elimDeg12(t, marked)

In [ ]:
(rand(5) .< 0.6) + 0

In [ ]:
elims1

In [ ]:
elims2

In [ ]:
ind = find(marked.<2)

In [ ]:
ts = t[ind,ind];
lt = lap(t)
lt[1,1] = lt[1,1] + 1
lts = lap(ts)
lts[1,1] = lts[1,1] + 1
b = randn(n);


In [ ]:
cumb = copy(b)

for i in 1:length(elims1)
    cumb[elims1[i].parent] += cumb[elims1[i].nodeid]
end
cumbs = cumb[ind]
sum(cumbs)

In [ ]:
xs = lts \ cumbs
norm(lts*xs-cumbs)

In [ ]:
x = zeros(n)
x[ind] = xs
for i in length(elims1):-1:1
    node = elims1[i].nodeid
    x[node] = x[elims1[i].parent] + cumb[node]/elims1[i].wtDeg
end
norm(lt*x-b)

In [ ]:
y = la \ b;
norm(la*y-b)
[y[ind] x[ind] xs]

In [ ]:
sum(las)

In [ ]:
function recTest(t,b, marked)

    n = length(b)

    elims1 = elimDeg1(t, marked)

    cumb = copy(b)

    for i in 1:length(elims1)
        cumb[elims1[i].parent] += cumb[elims1[i].nodeid]
    end


    ind = find(marked.<2)
    ts = t[ind,ind];
    lts = lap(ts)

    cumbs = cumb[ind]
    xs = pinv(full(lts))*cumbs
    
    x = zeros(n)
    x[ind] = xs
    for i in length(elims1):-1:1
        node = elims1[i].nodeid
        x[node] = x[elims1[i].parent] + cumb[node]/elims1[i].wtDeg
    end

    return x
    
end


In [ ]:
mkd = rand(0:1,n)
#elims1 = elimDeg1(t, mkd)
#ind = find(mkd .< 2)

In [ ]:
cumb = copy(b)

for i in 1:length(elims1)
    cumb[elims1[i].parent] += cumb[elims1[i].nodeid]
end
cumbs = cumb[ind]
sum(cumbs)

In [ ]:
sum(cumbs)

In [ ]:
b = randn(n)
b = b - mean(b)
lt = lap(t)
x = pinv(full(lt)) * b
norm(lt*x-b)

In [ ]:
x = recTest(t, b, mkd)
norm(lt*x-b)

In [ ]:
sum(mkd.==2)

In [ ]:
norm(b)

In [ ]:
function makeElimList(t)
    tr = matToTree(t)
    n = size(tr.children,1)  
    
    elims = Array{elimTreeNode}(0)
    for vi in n:-1:2
        v = tr.children[vi]
        push!(elims,elimTreeNode(v,tr.parent[v],tr.weights[vi]))
    end
    
    return elims
end

In [ ]:
function solveTree(elims,b)
    cumb = copy(b)
    n = size(b,1)
    for i in 1:(n-1)
        cumb[elims[i].parent] += cumb[elims[i].nodeid]
    end
    x = zeros(Float64,n)
    for i in (n-1):-1:1
        node = elims[i].nodeid
        x[node] = x[elims[i].parent] + cumb[node]/elims[i].wtDeg
    end
    return x
    
end

In [ ]:
@time el = makeElimList(t);

In [ ]:
lt = lap(t)
lt[end,end] += 1
@time F = factorize(lt)

In [ ]:
b = randn(size(t,1));
b = b - mean(b);
@time x = solveTree(el,b);
x = x - mean(x);
norm(lap(t)*x-b)

In [ ]:
@time y = F \ b;
norm(lt*y-b)

In [ ]:
elims = Array{elimTreeNode}(0)

In [ ]:
for vi in n:-1:2
    v = tr.children[vi]
    push!(elims,elimTreeNode(v,tr.parent[v],tr.weights[vi]))
end

In [ ]:
elims

In [ ]:
b = randn(n)
b = b - mean(b)

In [ ]:
cumb = copy(b)
for i in 1:(n-1)
    cumb[elims[i].parent] += cumb[elims[i].nodeid]
end
cumb

In [ ]:
x = zeros(Float64,n)
for i in (n-1):-1:1
    node = elims[i].nodeid
    x[node] = x[elims[i].parent] + cumb[node]/elims[i].wtDeg
end
x

In [ ]:
xs = pinv(full(lt))*b

In [ ]:
x-xs

In [ ]:
x = x - mean(x)

In [ ]:
lt*x - b

In [ ]:
b

In [ ]:
tr.weights

In [ ]:
st = compStretches(t,a);
aveStretch = sum(st)/nnz(a)

In [ ]:
rest = a-t;
fac = aveStretch*log(size(a,1))*2
heavy = rest+fac*t;

In [ ]:
fac

Sample with probability prop to stretch, but fix the number of edges we get to n/5

In [ ]:
n = size(a,1);

In [ ]:
strest = compStretches(t,rest)
probs = triu(strest);
probs = probs * (n/5)/ sum(probs);


In [ ]:
(pi,pj,pv) = findnz(probs)

In [ ]:
select = rand(size(pv)) .< pv;
sum(select)

In [ ]:
(ai,aj,av) = findnz(triu(rest))

In [ ]:
samp1 = sparse(ai,aj,av.*select./pv,n,n)
samp1 = samp1 + samp1';

In [ ]:
st1 = compStretches(t*fac, samp1);

In [ ]:
(s1i,s1j,s1v) = findnz(triu(samp1))

In [ ]:
maximum(x)

In [ ]:
a = chimera(100000,103+1111);
testGraph(a);


In [ ]:
include("../devel/kmpTest.jl")

In [ ]:
testAtSize(10000)

In [ ]:
x = testKMP(a,frac1 = 1/5, frac2 = 1/10)

In [ ]:
    (pi,pj,pv) = findnz(probs)
    select = rand(size(pv)) .< pv;
    (ai,aj,av) = findnz(triu(rest))
    
    samp1 = sparse(ai,aj,av.*select./pv,n,n)
#    samp1 = samp1 + samp1';
    
#    st1 = compStretches(t*fac, samp1);
#    (s1i,s1j,s1v) = findnz(triu(samp1))

In [ ]:
ringGraph(1)

In [ ]:
a = wtedChimera(1000000)
@time t = akpw(a);


In [ ]:
n = size(t,1)
nnz(a)

In [ ]:
@time ord = Laplacians.dfsOrder(t);

In [ ]:
@time aord = symperm(a,ord);
@time aord = aord + aord';

In [ ]:
pe = randperm(n)
@time ape = symperm(a,pe)
@time ape = ape + ape';

In [ ]:
x = randn(n)
@time b1 = ape*x;
@time b2 = aord*x;

In [ ]:
@show nnz(ape)
@show nnz(aord)

In [ ]:
tord = t[ord,ord];

In [ ]:
@time st1 = compStretches(tord,aord);

In [ ]:
@time st2 = compStretchesDFS(tord,aord);

In [ ]:
sum(abs(st1-st2))

In [ ]:
@time tr, d1 = matToTreeDepth(tord);

In [ ]:
@time d2 = treeDepthDFS(tord);

In [ ]:
norm(d1-d2)

In [ ]:
ord[1:10]

In [ ]:
tord[:,1000]

In [ ]:
function dfsOrder2(t::SparseMatrixCSC)
    n = size(t,1)
    seen = zeros(Bool,n)
    ord = Array{Int64}(0)
    stk = Array{Int64}(0)
    push!(stk,1)
    while ~isempty(stk)
        u = pop!(stk)
        push!(ord,u)
        seen[u] = true
        for vi in t.colptr[u]:(t.colptr[u+1]-1)
            v = t.rowval[vi]
            if !seen[v]
                push!(stk,v)
            end
        end
    end
    return ord

end

In [ ]:
ord = dfsOrder(tr);

In [ ]:
stack = Array{Int64}(0)

In [ ]:
~isempty(stack)

In [ ]:
function dfsOrder2(tr)
    ord = Array{Int64}(0)
    stack = Array{Int64}(0)
    push!(stack,tr.root)
    while ~isempty(stack)
        u = pop!(stack)
        push!(ord,u)
        for  vi in tr.kidsPtr[u]:(tr.numKids[u] + tr.kidsPtr[u] - 1)
            v = tr.children[vi]
            push!(stack,v)
        end
    end
    return ord

end

In [ ]:
ord2 = dfsOrder2(tr);

In [ ]:
norm(ord - ord2[size(tr.children,1):-1:1])

In [ ]:
# n = 100000
a = grid2(1000)
t = akpw(a)
aves = sum(compStretches(t,a))/nnz(a)

In [ ]:
avc = zeros(10)
for i in 1:10
    ap = randperm(a)
    tp = akpw(ap)
    avc[i] = sum(compStretches(tp,ap))/nnz(a)
end
mean(avc)

In [ ]:
avc

In [ ]:
?akpw!

In [ ]:
a = grid2(700)
@time t = akpw(a,exponentialX=false)
@show sum(compStretches(t,a))/nnz(a)
@time t = akpw(a,exponentialX=true)
@show sum(compStretches(t,a))/nnz(a);

In [ ]:
nits = 100
df = Array{Float64}(0)
dt = Array{Float64}(0)
timef = Array{Float64}(0)
timet = Array{Float64}(0)
for i in 1:nits
    a = wtedChimera(500000)
    tic();
    t = akpw(a,exponentialX=false)
    push!(timef,toc())
    push!(df,sum(compStretches(t,a))/nnz(a))
    
    tic();
    t = akpw(a,exponentialX=true)
    push!(timet,toc())
    push!(dt,sum(compStretches(t,a))/nnz(a))
    
    a = unweight(a)
    tic();
    t = akpw(a,exponentialX=false)
    push!(timef,toc())
    push!(df,sum(compStretches(t,a))/nnz(a))
    
    tic();
    t = akpw(a,exponentialX=true)
    push!(timet,toc())
    push!(dt,sum(compStretches(t,a))/nnz(a))
end
    


In [ ]:
x = collect(1:length(dt));
plot(x,sort(timet),"rx",x,sort(timef),"bo")

In [ ]:
plot(x,sort(timet./timef),"rx",x,sort(timef./timet),"bo")

In [ ]:
plot(x,sort(dt./df),"rx",x,sort(df./dt),"bo")

In [ ]:
mean(df)

In [ ]:
mean(dt)

In [ ]:
plot(x,sort(df),"rx",x,sort(dt),"bo")

In [ ]:
mean(timet)

In [ ]:
mean(df)

In [ ]:
mean(dt)

In [ ]:
function testSampler(a; t=akpw(a), frac1=1/5)


    n = size(a,1);

    rest = a-t;

    st = compStretches(t,rest);
    aveStretch = sum(st)/nnz(rest)
    @show aveStretch
    

    fac = aveStretch*log(size(a,1))*3
    heavy = rest+fac*t;

    (ai,aj,av) = findnz(triu(rest))
    (si,sj,sv) = findnz(triu(st))
    sv = sv ./ fac

    restijvs = restIJVS(ai,aj,av,sv)

    sampijvs = stretchSample2(restijvs,frac1)
    
    samp1 = sparse(sampijvs.i,sampijvs.j,sampijvs.v,n,n)
    samp1 = samp1 + samp1';
    
    st1 = compStretches(t*fac, samp1);
    (s1i,s1j,s1v) = findnz(triu(samp1))

    add = speye(n)/10^6;
    e = eigs(lap(heavy)+add,lap(samp1+t*fac)+add, nev=4);

    return maximum(e[1])
end


In [ ]:
a = grid2(50)
t = akpw(a)
frac1 = 2/3
    n = size(a,1);

    rest = a-t;

    st = compStretches(t,rest);
    aveStretch = sum(st)/nnz(rest)
    @show aveStretch
    

    fac = aveStretch*log(size(a,1))
    heavy = rest+fac*t;

    (ai,aj,av) = findnz(triu(rest))
    (si,sj,sv) = findnz(triu(st))
    sv = sv ./ fac

    restijvs = restIJVS(ai,aj,av,sv)

    sampijvs = stretchSample(restijvs,frac1)
    

In [ ]:
include("../src/KMPSolver.jl")
include("../devel/kmpTest.jl")

In [ ]:
n = 1000000
a = chimera(n)
la = lap(a)
b = randn(n);
b = b - mean(b);

t = akpw(a)
heavy = makeHeavy(a,t)
lheavy = lap(heavy);

In [ ]:
@profile f = KMPLapPrecon(a,t)
#@time xh = f(b)
#@show norm(lheavy*xh-b);

In [ ]:
Profile.clear()

In [ ]:
@time fa = lapWrapSolver(augTreeSolver,lheavy,tol=1e-2,maxits=1000)
@time xa = fa(b)
@show norm(lheavy*xa-b);

In [ ]:
function vecstats(s)
    println("length : ", size(s,1), ", min : ", minimum(s), ", mean : ", mean(s), ", max : ", maximum(s), ", sum : ", sum(s))
end

In [ ]:
vecstats(restijvs.s)

In [ ]:
a = grid2(200)
t = akpw(a)

frac1 = 1/5
    n = size(a,1);

    rest = a-t;

    st = compStretches(t,rest);
    aveStretch = sum(st)/nnz(rest)
    @show aveStretch
    

    targetStretch = 1/(2*log(n)/log(2))

In [ ]:
    fac = aveStretch/targetStretch
    heavy = rest+fac*t;

    (ai,aj,av) = findnz(triu(rest))

    (si,sj,sv) = findnz(triu(st))
    sv = sv ./ fac

    ijvs = IJVS(ai,aj,av,sv)

    ijvs1 = stretchSample(ijvs,targetStretch,frac1)
    ijvs2 = stretchSample(ijvs1,targetStretch,frac1)
    
    samp1 = sparse(ijvs1.i,ijvs1.j,ijvs1.v,n,n)
    samp1 = samp1 + samp1';
    
    samp2 = sparse(ijvs2.i,ijvs2.j,ijvs2.v,n,n)
    samp2 = samp2 + samp2';

In [ ]:
    add = speye(n)/10^6;
@time e = eigs(lap(heavy)+add,lap(samp1+t*fac)+add, nev=1, maxiter = 10);
@show maximum(e[1])
@time e = eigs(lap(samp1)+add,lap(samp2+t*fac)+add, nev=1, maxiter = 10);
@show maximum(e[1])

In [ ]:
vecstats(ijvs.s)
vecstats(ijvs1.s)
vecstats(ijvs2.s)

In [ ]:
targetStretch

In [ ]:
vecstats(sampijvs.s)

In [ ]:
mean(sampijvs.s)

In [ ]:
maximum(sampijvs.s)

In [ ]:
length(sampijvs.s)

In [ ]:
s2 = stretchSampleInner(sampijvs,1/2)
vecstats(s2.s)

In [ ]:
sum(s2.s .< 0.2)

In [ ]:
1/log(size(a,1))

In [ ]:
mean(restijvs.s)

In [ ]:
s3 = stretchSampleInner(s2,1/5)
[mean(s3.s), maximum(s3.s)]

In [ ]:
size(s3.i,1)